# Network in Network

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_SERVER_URI = "http://localhost:20000"
MLFLOW_REGISTRY_SERVER_URI = "http://localhost:20000"
mlflow_client = MlflowClient(MLFLOW_TRACKING_SERVER_URI, MLFLOW_REGISTRY_SERVER_URI)

In [3]:
from mlflow.entities import Experiment

import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_SERVER_URI)
mlflow.set_registry_uri(MLFLOW_REGISTRY_SERVER_URI)

experiment: Experiment = mlflow.set_experiment(experiment_name="NiN 01")

In [4]:
run = mlflow_client.create_run(
    experiment_id=experiment.experiment_id,
)

In [5]:
mlflow.start_run(run.info.run_id)

<ActiveRun: >

In [6]:
from recognizer.utils.constants import ROOT_DIR, TARGET_TO_ENCODING
from recognizer.utils.utils import get_metadata_from_filename

DATASET_DIR_POSTA = ROOT_DIR / "data" / "all-20percent"

In [7]:
BATCH_SIZE = 8
NUM_CLASSES = 64
EPOCHS = 30
NUM_FRAMES = 8

LR = 0.005

mlflow_client.log_param(run_id=run.info.run_id, key="BATCH_SIZE", value=BATCH_SIZE)
mlflow_client.log_param(run_id=run.info.run_id, key="NUM_CLASSES", value=NUM_CLASSES)
mlflow_client.log_param(run_id=run.info.run_id, key="EPOCHS", value=EPOCHS)
mlflow_client.log_param(run_id=run.info.run_id, key="NUM_FRAMES", value=NUM_FRAMES)
mlflow_client.log_param(run_id=run.info.run_id, key="LR", value=LR)

0.005

## Load Data

In [8]:
import os

import pandas as pd

targets = []
subjects = []
repetitions = []
files = []

for file in os.listdir(DATASET_DIR_POSTA):

    target, subject, repetition = get_metadata_from_filename(file)

    targets.append(target)
    subjects.append(subject)
    repetitions.append(repetition)
    files.append(str((DATASET_DIR_POSTA / file).resolve()))


metadata = pd.DataFrame(
    data={
        "target": targets,
        "subject": subjects,
        "repetition": repetitions,
        "file": files,
    }
)

metadata["target_encoding"] = metadata["target"].map(TARGET_TO_ENCODING)

metadata.head()

,target,subject,repetition,file,target_encoding
0,001,001,001,C:\Users\facun\Desktop\DEV\ecd-trabajo-final\d...,0
1,001,001,002,C:\Users\facun\Desktop\DEV\ecd-trabajo-final\d...,0
2,001,001,003,C:\Users\facun\Desktop\DEV\ecd-trabajo-final\d...,0
3,001,001,004,C:\Users\facun\Desktop\DEV\ecd-trabajo-final\d...,0
4,001,001,005,C:\Users\facun\Desktop\DEV\ecd-trabajo-final\d...,0


### Train/Test split

In [9]:
import numpy as np

size = 1
replace = False
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]

testing_set = metadata.groupby(["target", "subject"], as_index=False).apply(fn)

testing_set.index = testing_set.index.droplevel(0)

training_set = metadata.loc[~metadata.index.isin(testing_set.index), :]

### Pre-processing functions

In [10]:
def transform(x):
    """Permutes the element to match the format expected by PyTorch: (C<channels>, T<frames>, H<height>, W<width>)"""
    # Transpose video from (T<frames>, H<height>, W<width>, C<channels>) to (C<channels>, T<frames>, H<height>, W<width>)
    return x.permute(3, 0, 1, 2).float()


### Datasets

In [11]:
from recognizer.dataset import SampledVideoDataset

training_dataset = SampledVideoDataset(
    video_filenames=training_set["file"].values,
    labels=training_set["target_encoding"].values,
    num_frames=NUM_FRAMES,
    transform=transform,
)

testing_dataset = SampledVideoDataset(
    video_filenames=testing_set["file"].values,
    labels=testing_set["target_encoding"].values,
    num_frames=NUM_FRAMES,
    transform=transform,
)

print(f"Training/testing set: ({len(training_dataset)}, {len(testing_dataset)})")

Training/testing set: (2560, 640)


### Data Loaders

In [12]:
import torch 

from torch import nn

from recognizer.models.nin import NiNVideoClassifier

# Las imágenes del video son de 384x216 
# Hay 8 frames por video

# Es decir que cada elemento inut será de 3x8x216x382 (C<channels> * T<frames> * H<height> * W<width>)
# Eso es un total de 

model = NiNVideoClassifier(
    num_classes=NUM_CLASSES,
)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle = True)

test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=BATCH_SIZE, shuffle = False)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
model.to(device)

device

'cuda'

In [13]:
p = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Params: {p}")

Params: 3419776


## Training

In [14]:
import subprocess

CMD = '''
on run argv
  display notification (item 2 of argv) with title (item 1 of argv) sound name "Glass"
end run
'''

def notify(title, text):
    subprocess.call(['osascript', '-e', CMD, title, text])

In [15]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

cm = None

metrics = {
    "training_loss": [],
    "testing_loss": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": []
}

In [16]:
print(
f"""
Beginning model training with parameters:
- Epochs: {EPOCHS}
- Batch Size: {BATCH_SIZE}
"""
)

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1} - Training")

    model.train()

    running_training_loss = 0.0

    for _, data in enumerate(train_loader):

        batch, labels = data[0].to(device), data[1].to(device)

        logits = model(batch)

        loss = loss_function(logits, labels)

        running_training_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_training_loss = running_training_loss / len(train_loader)
    metrics["training_loss"].append(round(average_training_loss, 2))

    print(f"Epoch {epoch + 1} - AVG Training Loss: {average_training_loss:.2f}")

    # Evaluation
    print(f"Epoch {epoch + 1} - Evaluation")

    model.eval()

    all_preds = []
    all_targets = []

    running_testing_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)

            logits = model(inputs)

            _, preds = torch.max(logits, 1)

            all_preds.extend(preds.detach().cpu())
            all_targets.extend(labels.detach().cpu())

            loss = loss_function(logits, labels)
            running_testing_loss += loss.item()

    average_testing_loss = running_testing_loss / len(test_loader)
    metrics["testing_loss"].append(round(average_testing_loss, 2))

    print(f"Epoch {epoch + 1} - AVG Testing Loss: {average_testing_loss:.2f}")

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    metrics["accuracy"].append(accuracy_score(all_targets, all_preds))
    metrics["precision"].append(precision_score(all_targets, all_preds, average="macro"))
    metrics["recall"].append(recall_score(all_targets, all_preds, average="macro"))
    metrics["f1"].append(f1_score(all_targets, all_preds, average="macro"))

    for metric in metrics.keys():
        mlflow_client.log_metric(
            run_id=run.info.run_id,
            key=metric,
            value=metrics[metric][epoch],
            step=epoch,
        )

    if epoch == EPOCHS - 1:
        cm = confusion_matrix(all_targets, all_preds)

    print(f"""
Epoch {epoch + 1}:
    - Accuracy  : {metrics['accuracy'][-1]}
    - Precision : {metrics['precision'][-1]}
    - Recall    : {metrics['recall'][-1]}
    - F1        : {metrics['f1'][-1]}
    """)


Beginning model training with parameters:
- Epochs: 30
- Batch Size: 8

Epoch 1 - Training
Epoch 1 - AVG Training Loss: 4.19
Epoch 1 - Evaluation
Epoch 1 - AVG Testing Loss: 4.16


c:\Users\facun\AppData\Local\pypoetry\Cache\virtualenvs\recognizer-U-rlUTHu-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 1:
    - Accuracy  : 0.015625
    - Precision : 0.000244140625
    - Recall    : 0.015625
    - F1        : 0.0004807692307692308
    
Epoch 2 - Training
Epoch 2 - AVG Training Loss: 4.16
Epoch 2 - Evaluation
Epoch 2 - AVG Testing Loss: 4.16

Epoch 2:
    - Accuracy  : 0.015625
    - Precision : 0.000244140625
    - Recall    : 0.015625
    - F1        : 0.0004807692307692308
    
Epoch 3 - Training


c:\Users\facun\AppData\Local\pypoetry\Cache\virtualenvs\recognizer-U-rlUTHu-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 3 - AVG Training Loss: 4.16
Epoch 3 - Evaluation
Epoch 3 - AVG Testing Loss: 4.16

Epoch 3:
    - Accuracy  : 0.015625
    - Precision : 0.000244140625
    - Recall    : 0.015625
    - F1        : 0.0004807692307692308
    
Epoch 4 - Training


...

Epoch 5 - AVG Training Loss: 4.16
Epoch 5 - Evaluation
Epoch 5 - AVG Testing Loss: 4.16

Epoch 5:
    - Accuracy  : 0.015625
    - Precision : 0.000244140625
    - Recall    : 0.015625
    - F1        : 0.0004807692307692308
    
Epoch 6 - Training


c:\Users\facun\AppData\Local\pypoetry\Cache\virtualenvs\recognizer-U-rlUTHu-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 6 - AVG Training Loss: 4.16
Epoch 6 - Evaluation
Epoch 6 - AVG Testing Loss: 4.16

Epoch 6:
    - Accuracy  : 0.015625
    - Precision : 0.000244140625
    - Recall    : 0.015625
    - F1        : 0.0004807692307692308
    
Epoch 7 - Training


c:\Users\facun\AppData\Local\pypoetry\Cache\virtualenvs\recognizer-U-rlUTHu-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

## Evaluation

### Confusion Matrix

In [ ]:
import seaborn as sns

sns.heatmap(data=cm)

### Accuracy/Recall/Precision

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(8,4))

ax.plot(metrics["accuracy"], label="Accuracy", marker=".")
ax.plot(metrics["precision"], label="Precision", marker=".")
ax.plot(metrics["recall"], label="Recall", marker=".")
ax.plot(metrics["f1"], label="F1", marker=".")

ax.set_xticks(range(0, epoch+1))
ax.set_xticklabels(range(1, epoch+2))
ax.set_yticks(np.arange(0, 1.1, 0.1))
ax.set_yticklabels(np.arange(0, 1.1, 0.1))
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(alpha=0.1)
plt.show()

### Loss

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(8,4))

ax.plot(metrics["training_loss"], label="Training Loss", marker=".", color="steelblue")
ax.plot(metrics["testing_loss"], label="Evaluation Loss", marker=".", color="orange")

ax.set_xticks(range(0, epoch+1))
ax.set_xticklabels(range(1, epoch+2))
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(alpha=0.1)
plt.show()

## Save model

In [ ]:
from recognizer.utils.constants import ROOT_DIR

MODELS_DIR = ROOT_DIR / "models"

MODEL_PATH = MODELS_DIR / "nin.pth"

with open(MODEL_PATH, "w") as f:
        f.write("")

torch.save(model, f=MODEL_PATH)

In [ ]:
mlflow_client.log_artifact(
    run_id=run.info.run_id,
    local_path=str(MODEL_PATH),
    artifact_path="model.pth",
)